In [1]:
%pip install torch datasets evaluate transformers sympy accelerate peft bitsandbytes

In [5]:
from datasets import load_dataset
import os

# Suppose your CSV is named 'checkworthy_data.csv' with columns: "text", "label"
train_path = os.path.join(os.getcwd(), "cleaned_data_train.csv")
test_path = os.path.join(os.getcwd(), "cleaned_data_test.csv")
data_files = {"train": train_path, "test": test_path}  # or separate train/test if you have them
raw_datasets = load_dataset("csv", data_files=data_files)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def tokenize_function(examples):
    texts = [str(text) for text in examples["text"]]
    return tokenizer(texts, padding="max_length", truncation=True)

In [7]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/8980 [00:00<?, ? examples/s]

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [10]:
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,                 # rank
    lora_alpha=32,       # scaling
    lora_dropout=0.05,   # dropout in LoRA
    bias="none",         # no bias for LoRA
    task_type="SEQ_CLS",
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"]
)


In [14]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)
print("Created LoRA model. Trainable parameters:",
      sum(p.numel() for p in model.parameters() if p.requires_grad))


Created LoRA model. Trainable parameters: 887042


In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", learning_rate = 0.00002, num_train_epochs=3, per_device_train_batch_size=16, per_device_eval_batch_size=16)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saximn (saximn-nanyang-technological-university-singapore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.004600,0.006343,0.998886
2,0.002600,0.004059,0.999220
3,0.001900,0.004060,0.999109


TrainOutput(global_step=6735, training_loss=0.016791744207729654, metrics={'train_runtime': 4675.2018, 'train_samples_per_second': 23.048, 'train_steps_per_second': 1.441, 'total_flos': 1.456752097308672e+16, 'train_loss': 0.016791744207729654, 'epoch': 3.0})

In [21]:
# Save the model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json')

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_model")
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_model")

In [ ]:
import pandas as pd

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train["label"].value_counts(normalize=True)

,proportion
label,
0,0.657875
1,0.342125


In [ ]:
test["label"].value_counts(normalize=True)

In [22]:
from sklearn.metrics import classification_report
import numpy as np

predictions = trainer.predict(tokenized_datasets["test"])
predicted_labels = np.argmax(predictions.predictions, axis=1)


In [23]:
true_labels = tokenized_datasets['test']['label']

# Generate a classification report
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4284
           1       1.00      1.00      1.00      4696

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

